In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

from neurobiases import (TriangularModel,
                         EMSolver,
                         TCSolver,
                         solver_utils,
                         plot)
from sklearn.model_selection import check_cv

%matplotlib inline

In [ ]:
files = ['em_sparse.h5', 'tc_sparse.h5']

In [ ]:
results = [h5py.File(file, 'r') for file in files]

In [ ]:
# Print out entries in H5 files
for file, result in zip(files, results):
    print(file)
    print(list(result))
    print('---')

In [ ]:
# Print out shape keys
for file, result in zip(files, results):
    print(file)
    print(result['shape_key'][:])
    print('---')

In [ ]:
# Print out actual seeds for generating parameters/data
for file, result in zip(files, results):
    print(file)
    print(result['shape_key'][:])
    print(result['a_true'].shape)
    print(result['b_true'].shape)
    print('---')

In [ ]:
# Print out seeds
for file, result in zip(files, results):
    print(file)
    print(result.attrs['coupling_rng'])
    print(result.attrs['tuning_rng'])
    print(result.attrs['dataset_rng'])
    print(result.attrs['fitter_rng'])
    print('---')

In [ ]:
# Print out actual seeds for generating parameters/data
for file, result in zip(files, results):
    print(file)
    print(result['coupling_rngs'][:])
    print(result['tuning_rngs'][:])
    print(result['dataset_rngs'][:])
    print('---')

In [ ]:
# Print out actual seeds for generating parameters/data
for file, result in zip(files, results):
    print(file)
    tm = TriangularModel(
        model='linear',
        parameter_design='direct_response',
        M=result.attrs['M'],
        N=result.attrs['N'],
        K=result.attrs['K'],
        corr_cluster=result.attrs['corr_cluster'],
        corr_back=result.attrs['corr_back'],
        coupling_distribution=result.attrs['coupling_distribution'],
        coupling_sparsity=result.attrs['coupling_sparsity'],
        coupling_loc=result['coupling_locs'][1],
        coupling_scale=result.attrs['coupling_scale'],
        coupling_rng=result['coupling_rngs'][1],
        tuning_distribution=result.attrs['tuning_distribution'],
        tuning_loc=result['tuning_locs'][0],
        tuning_scale=result.attrs['tuning_scale'],
        tuning_sparsity=result.attrs['tuning_sparsity'],
        tuning_rng=result['tuning_rngs'][1]
    )
    print(tm.a.ravel())
    print(result['a_true'][0, 1, 1, 0, 1, 0, 0])
    print(result['a_true'][0, 1, 1, 0, 1, 0, 1])
    print(tm.b.ravel())
    print(result['b_true'][0, 1, 1, 0, 1, 0, 0])
    print(result['b_true'][0, 1, 1, 0, 1, 0, 1])
    print('---')


In [ ]:
# Print out actual seeds for generating parameters/data
for file, result in zip(files, results):
    print(file)
    tm = TriangularModel(
        model='linear',
        parameter_design='direct_response',
        M=result.attrs['M'],
        N=result.attrs['N'],
        K=result.attrs['K'],
        corr_cluster=result.attrs['corr_cluster'],
        corr_back=result.attrs['corr_back'],
        coupling_distribution=result.attrs['coupling_distribution'],
        coupling_sparsity=result.attrs['coupling_sparsity'],
        coupling_loc=result['coupling_locs'][0],
        coupling_scale=result.attrs['coupling_scale'],
        coupling_rng=result['coupling_rngs'][0],
        tuning_distribution=result.attrs['tuning_distribution'],
        tuning_loc=result['tuning_locs'][0],
        tuning_scale=result.attrs['tuning_scale'],
        tuning_sparsity=result.attrs['tuning_sparsity'],
        tuning_rng=result['tuning_rngs'][0]
    )
    X, Y, y = tm.generate_samples(n_samples=result.attrs['D'], rng=result['dataset_rngs'][0])
    cv = check_cv(2)

    train_idx, test_idx = list(cv.split(X))[0]
    X_train = X[train_idx]
    Y_train = Y[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    Y_test = Y[test_idx]
    y_test = y[test_idx]

    if 'em' in file:
        solver = EMSolver(
            X=X_train,
            Y=Y_train,
            y=y_train,
            K=1,
            c_coupling=np.logspace(-3, -1, 2)[0],
            c_tuning=np.logspace(-3, -1, 2)[0],
            tol=result.attrs['tol'],
            max_iter=result.attrs['max_iter'],
            initialization=result.attrs['initialization'],
            solver=result.attrs['solver'],
            rng=result.attrs['fitter_rng'],
            fa_rng=2332).fit_em(refit=True, verbose=True)
        print(solver.a.ravel())
        print(result['a_est'][0, 0, 0, 0, 0, 0, 0])
        print(solver.b.ravel())
        print(result['b_est'][0, 0, 0, 0, 0, 0, 0])
    elif 'tc' in file:
        solver = TCSolver(
            X=X_train,
            Y=Y_train,
            y=y_train,
            c_coupling=np.logspace(1, 4, 3)[0],
            c_tuning=np.logspace(1, 4, 3)[1],
            tol=result.attrs['tol'],
            max_iter=result.attrs['max_iter'],
            initialization=result.attrs['initialization'],
            solver=result.attrs['solver'],
            rng=result.attrs['fitter_rng']).fit_lasso(refit=True)
        print(solver.a.ravel())
        print(result['a_est'][0, 1, 1, 0, 1, 0, 1])
        print(solver.b.ravel())
        print(result['b_est'][0, 1, 1, 0, 1, 0, 1])


In [ ]:
np.logspace(-3, -1, 2)[0]

In [ ]:
results[0].attrs['tol']

In [ ]:
np.logspace(0, 1, 2)[1]

In [ ]:
np.logspace(-3, -1, 2)